In [16]:
import pandas as pd
import numpy as np
# import trading_strategy_utils as tsu
import update_dataframe as ud
import grid_search as gs

In [17]:
# Load the ada data
ada_df = pd.read_csv('data/ADA.csv', parse_dates=['ts_hour'])

# Select March in 2022
ada_df_march_22 = ada_df[(ada_df['ts_hour'].dt.month == 3) & (ada_df['ts_hour'].dt.year == 2022)]
ada_df_march_22 = ada_df_march_22[['ts_hour', 'pnl']]

ada_df_march_22.head()

,ts_hour,pnl
32070,2022-03-01 00:00:00,-0.000542
32071,2022-03-01 01:00:00,0.006158
32072,2022-03-01 02:00:00,0.001905
32073,2022-03-01 03:00:00,0.000959
32074,2022-03-01 04:00:00,0.001538


In [18]:
ada_df_march_22.tail()

,ts_hour,pnl
32809,2022-03-31 19:00:00,0.004157
32810,2022-03-31 20:00:00,-0.003229
32811,2022-03-31 21:00:00,-0.003488
32812,2022-03-31 22:00:00,0.004522
32813,2022-03-31 23:00:00,0.009128


### Grid Search for EMA(Short & Long) params

In [19]:
# Define the range of parameters for grid search
short_window_range = range(4, 7, 1)
long_window_range = range(11, 15, 1)


# Perform grid search
grid_search_march_19 = gs.grid_search_2_params(ada_df_march_22, short_window_range, long_window_range)
grid_search_march_19_sorted = grid_search_march_19.sort_values('total_weighted_pnl', ascending=False)

short_window: 4, long_window: 11, total_weighted_pnl: 0.060243148426606365
short_window: 4, long_window: 12, total_weighted_pnl: 0.05255054196038957
short_window: 4, long_window: 13, total_weighted_pnl: 0.04713187747224316
short_window: 4, long_window: 14, total_weighted_pnl: 0.05931250391495042
short_window: 5, long_window: 11, total_weighted_pnl: 0.062034892179573836
short_window: 5, long_window: 12, total_weighted_pnl: 0.06388126203822175
short_window: 5, long_window: 13, total_weighted_pnl: 0.058111287089250264
short_window: 5, long_window: 14, total_weighted_pnl: 0.059178284611922795
short_window: 6, long_window: 11, total_weighted_pnl: 0.0585388338214334
short_window: 6, long_window: 12, total_weighted_pnl: 0.07214309393339391
short_window: 6, long_window: 13, total_weighted_pnl: 0.07252714156332138
short_window: 6, long_window: 14, total_weighted_pnl: 0.0707058037473264
Grid search finished.


In [20]:
grid_search_march_19_sorted

,short_window,long_window,total_weighted_pnl
10,6,13,0.072527
9,6,12,0.072143
11,6,14,0.070706
5,5,12,0.063881
4,5,11,0.062035
0,4,11,0.060243
3,4,14,0.059313
7,5,14,0.059178
8,6,11,0.058539
6,5,13,0.058111


# Run the trading strategy with the best EMA(Short & Long) params found by grid search

In [21]:
best_short_window = grid_search_march_19_sorted.iloc[0]['short_window']
best_long_window = grid_search_march_19_sorted.iloc[0]['long_window']
# best_short_window = 6
# best_long_window = 13
true_weight = 1

print('Best short window:', best_short_window)
print('Best long window:', best_long_window)
# Create a new DataFrame to store results with the best parameters
strategy_result = pd.DataFrame(columns=['ts_hour', 'pnl', 'EMA_short', 'EMA_long', 'is_positive', 'signal', 'count', 'consistency', 'is_continuous', 'class', 'weight_percentage', 'weight', 'weighted_pnl'])
results_list = []

# Update the DataFrame with the best parameters
for _, row in ada_df_march_19.iterrows():
    strategy_result = ud.update_dataframe_with_new_row(strategy_result, row, results_list, short_window=best_short_window, long_window=best_long_window, true_weight=true_weight)
print("Done")

Best short window: 6.0
Best long window: 13.0
Done


In [22]:
strategy_result[10:20].head(10)

,ts_hour,pnl,EMA_short,EMA_long,is_positive,signal,count,consistency,is_continuous,class,weight_percentage,weight,weighted_pnl
10,2019-03-01 10:00:00,-0.001433,-0.000428,-2.447621e-04,0,-1,0,NaN,NaN,0,0,0,0
11,2019-03-01 11:00:00,-0.000234,-0.000372,-2.432216e-04,0,0,0,NaN,NaN,0,0,0,0
12,2019-03-01 12:00:00,-0.000538,-0.000419,-2.852679e-04,0,0,0,NaN,0,0,0,0.0,-0.0
13,2019-03-01 13:00:00,-0.000112,-0.000332,-2.605362e-04,0,0,0,NaN,0,0,0,0.0,-0.0
14,2019-03-01 14:00:00,0.001561,0.000209,-2.615325e-07,1,1,1,"[13, 14]",1,1,100,1.0,0.0
15,2019-03-01 15:00:00,0.000311,0.000238,4.419640e-05,1,0,-1,"[13, 14, 15]",1,2.2,20,0.2,0.000311
16,2019-03-01 16:00:00,0.000455,0.000300,1.028143e-04,1,0,0,NaN,0,2.3,20,0.2,0.000091
17,2019-03-01 17:00:00,0.006419,0.002048,1.005183e-03,1,0,0,NaN,0,2.3,20,0.2,0.001284
18,2019-03-01 18:00:00,0.000697,0.001662,9.611323e-04,1,0,0,NaN,0,2.3,20,0.2,0.000139
19,2019-03-01 19:00:00,-0.000202,0.001130,7.950307e-04,0,0,0,NaN,0,0,0,0.0,-0.00004


In [23]:
strategy_result['class'].value_counts()

class
0      309
2.3    274
1       92
2.2     54
3.1     10
Name: count, dtype: int64

In [24]:
# import matplotlib.pyplot as plt
import plotly.graph_objects as go
import nbformat

from plotly.subplots import make_subplots

In [25]:
# Create the cumulative weighted PnL column
strategy_result_copy = strategy_result.copy()
strategy_result_copy['cumulative_weighted_pnl'] = strategy_result_copy['weighted_pnl'].cumsum()


In [26]:
# Create the plot
fig = go.Figure()

# Add the cumulative weighted PnL line
fig.add_trace(go.Scatter(
    x=strategy_result_copy['ts_hour'],
    y=strategy_result_copy['cumulative_weighted_pnl'],
    mode='lines',
    name='Cumulative Weighted PnL',
    line=dict(color='blue')
))

# Update layout
fig.update_layout(
    title='Cumulative Weighted PnL over Time with Annotations for Non-zero Classes',
    xaxis_title='Time',
    yaxis_title='Cumulative Weighted PnL',
    # template='plotly_white'
)


# Show the plot
fig.show()

In [27]:

# Define shape and color map for different classes
class_marker_map = {
    '1': {'color': 'red', 'symbol': 'circle', 'size': 10},
    '2.1': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '2.2': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '2.3': {'color': 'blue', 'symbol': 'triangle-up', 'size': 8},
    '3.1': {'color': 'green', 'symbol': 'square', 'size': 6},
    '3.2': {'color': 'green', 'symbol': 'square', 'size': 6},
    '3.3': {'color': 'green', 'symbol': 'square', 'size': 6}
}

# Create the plot
fig = go.Figure()

# Add the cumulative weighted PnL line
fig.add_trace(go.Scatter(
    x=strategy_result_copy['ts_hour'],
    y=strategy_result_copy['cumulative_weighted_pnl'],
    mode='lines',
    name='Cumulative Weighted PnL',
    line=dict(color='blue')
))

# Add Class and Weight points to the plot
for class_type in strategy_result_copy['class'].unique():
    if class_type not in class_marker_map:
        continue
    class_df = strategy_result_copy[strategy_result_copy['class'] == class_type]
    marker = class_marker_map[str(class_type)]
    fig.add_trace(go.Scatter(
        x=class_df['ts_hour'],
        y=class_df['cumulative_weighted_pnl'],
        mode='markers',
        name=f'Class {class_type}',
        marker=dict(color=marker['color'], symbol=marker['symbol'], size=marker['size']),
        text=[f"Class: {row['class']}<br>Weight: {row['weight']:.2f}" for index, row in class_df.iterrows()],
        hoverinfo='text'
    ))

# Update layout
fig.update_layout(
    title='Cumulative Weighted PnL over Time with Annotations for Non-zero Classes',
    xaxis_title='Time',
    yaxis_title='Cumulative Weighted PnL',
    # template='plotly_white'
)

# Show the plot
fig.show()

In [13]:
strategy_result.weighted_pnl.sum()

0.18672020048803822